In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pickle
import datetime

In [13]:
tokenizer = Tokenizer()
data = open('data.txt').read()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)
print(total_words)

{'the': 1, 'to': 2, 'and': 3, 'he': 4, 'of': 5, 'his': 6, 'was': 7, 'in': 8, 'it': 9, 'had': 10, 'that': 11, 'a': 12, 'as': 13, 'with': 14, 'gregor': 15, 'not': 16, 'she': 17, 'for': 18, 'him': 19, 'her': 20, 'would': 21, 'at': 22, 'but': 23, 'on': 24, 'they': 25, 'all': 26, 'this': 27, 'be': 28, 'from': 29, 'room': 30, 'if': 31, 'you': 32, 'or': 33, 'could': 34, 'out': 35, 'have': 36, 'there': 37, 'been': 38, "gregor's": 39, 'so': 40, 'father': 41, 'sister': 42, 'gutenberg': 43, 'now': 44, 'project': 45, 'himself': 46, 'door': 47, 'no': 48, 'any': 49, 'then': 50, 'up': 51, 'back': 52, 'even': 53, 'mother': 54, 'by': 55, 'into': 56, 'one': 57, 'work': 58, 'about': 59, 'did': 60, 'more': 61, 'their': 62, 'when': 63, 'what': 64, 'were': 65, 'them': 66, 'way': 67, 'only': 68, 'do': 69, 'time': 70, 'i': 71, 'other': 72, 'than': 73, 'tm': 74, 'is': 75, 'just': 76, 'said': 77, 'little': 78, 'without': 79, 'first': 80, '1': 81, 'get': 82, 'still': 83, 'see': 84, 'which': 85, 'some': 86, 'whil

In [14]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [6]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=15, verbose=1)
#print model.summary()
print(model.summary)

c:\Users\sanda\Desktop\OnGoing\RNN\rnn\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/15
 18/724 [..............................] - ETA: 33s - loss: 7.3261 - accuracy: 0.0330

KeyboardInterrupt: 

In [23]:
filename = "model"+str(datetime.datetime.today().strftime("%d%m%y"))
model.save("saved_model/"+filename)

INFO:tensorflow:Assets written to: saved_model/model171221\assets


INFO:tensorflow:Assets written to: saved_model/model171221\assets


In [11]:
modelname = 'saved_model/model171221'
model = tf.keras.models.load_model(modelname)
print(modelname+" loded.")
while True:
    seed_text = input("::")
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    classes=np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index == classes:
            output_word = word
            seed_text += " " + output_word
            print(seed_text)
            break
        else:
            print("no match found ...")
            break
    

saved_model/model171221 loded.
no match found ...
no match found ...
no match found ...


KeyboardInterrupt: Interrupted by user